In [6]:
%run web_scrape_func_def.ipynb

imports loaded.

> fighters_stats_scrape func:

    get_row(soup)
    get_record(soup)
    get_name(soup)
    get_stats(soup)
    get_columns(soup)
    get_html_data(soup)
    get_soup(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})
    get_links(soup)


# Main( )

In [12]:
url = 'http://ufcstats.com/statistics/fighters?char=b&page=all'
soup = get_soup(url)
get_links(soup)[0:3]

array(['http://ufcstats.com/fighter-details/013da757877044a2',
       'http://ufcstats.com/fighter-details/01641ba5df0c69b0',
       'http://ufcstats.com/fighter-details/01f022fa494188e2'],
      dtype='<U52')

In [27]:
link_list = []
for page in tqdm(ascii_letters.lower()):
    url = 'http://ufcstats.com/statistics/fighters?char=' + page + '&page=all'
    link_list.extend(get_links(soup=get_soup(url=url)))

print('%d fighters links loaded.'%len(link_list))    

100%|██████████| 52/52 [00:41<00:00,  1.24it/s]

7848 fighters links loaded.


In [33]:
data = np.array([columns])
data

array([['name', 'wins', 'losses', 'draws', 'Height:', 'Weight:',
        'Reach:', 'STANCE:', 'DOB:', 'SLpM:', 'Str. Acc.:', 'SApM:',
        'Str. Def:', '', 'TD Avg.:', 'TD Acc.:', 'TD Def.:',
        'Sub. Avg.:']], dtype='<U10')

In [35]:
for fighter_link in tqdm(link_list):
    row = get_row(soup=get_soup(url=fighter_link))
    data = np.vstack([data,row])

100%|██████████| 7848/7848 [1:15:46<00:00,  1.73it/s]


In [26]:
get_row(soup=get_soup(url='http://ufcstats.com/fighter-details/be4076c9e8d791e3'))

array(['Joao Zeferino', '21', '9', '0', '5\' 11"', '170 lbs.', '--',
       'Orthodox', 'Jan 15, 1986', '0.83', '36%', '2.60', '48%', '',
       '0.50', '5%', '50%', '1.0'], dtype='<U13')

In [ ]:
>>>>

### Extracting columns structure:

In [7]:
url = 'http://ufcstats.com/statistics/fighters?char=a&page=all'
soup = get_soup(url)
links = get_links(soup)
columns = get_columns( get_soup(links[1]) )
print(columns)

['name', 'wins', 'losses', 'draws', 'Height:', 'Weight:', 'Reach:', 'STANCE:', 'DOB:', 'SLpM:', 'Str. Acc.:', 'SApM:', 'Str. Def:', '', 'TD Avg.:', 'TD Acc.:', 'TD Def.:', 'Sub. Avg.:']


### constructing pandas data frame

In [8]:
df = pd.DataFrame(columns = columns)
df

,name,wins,losses,draws,Height:,Weight:,Reach:,STANCE:,DOB:,SLpM:,Str. Acc.:,SApM:,Str. Def:,,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:


### extracting fighters statistics

In [9]:
pages = ascii_letters.lower()

pages_amount = len(pages)
current_page = 0

for page in tqdm(pages):
    current_page += 1 
    url = 'http://ufcstats.com/statistics/fighters?char=' + page + '&page=all'
    #soup = get_soup(url)
    page_links = get_links(get_soup(url))
    links_amount = len(page_links)
    
    current_link = 0
    for stat_page in page_links:
        current_link += 1
        percent = round((((current_page-1)/pages_amount) + (current_link/links_amount)*(1/pages_amount)) * 100, 2)
        print('Scraping page %d/%d (%d/%d links) => %s %% completed ' %(current_page, pages_amount, current_link, links_amount, percent), end = '\r')
        stat_page_soup = get_soup(stat_page)
        row = get_row(stat_page_soup)
        df.loc[len(df)] = row
        

  0%|          | 0/52 [00:00<?, ?it/s]

  2%|▏         | 1/52 [01:46<1:30:12, 106.12s/it]

  2%|▏         | 1/52 [04:06<3:29:52, 246.91s/it]

Error on: http://ufcstats.com/fighter-details/e023fa2e648ea0f1


AttributeError: 'NoneType' object has no attribute 'find'

In [13]:
df.tail()

,name,wins,losses,draws,Height:,Weight:,Reach:,STANCE:,DOB:,SLpM:,Str. Acc.:,SApM:,Str. Def:,,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:
7839,Joao Zeferino,21,9,0,"5' 11""",170 lbs.,--,Orthodox,"Jan 15, 1986",0.83,36%,2.60,48%,,0.50,5%,50%,1.0
7840,Zhang Tiequan,15,4,0,"5' 8""",155 lbs.,"69""",Orthodox,"Jul 25, 1978",1.23,36%,2.14,51%,,1.95,58%,75%,3.4
7841,Carlos Zevallos,3,,-0,"6' 0""",205 lbs.,--,Orthodox,--,4.36,65%,2.28,68%,,0.00,0%,100%,0.0
7842,Zach Zane,10,7,0,"5' 7""",145 lbs.,"69""",Southpaw,"Dec 14, 1989",0.87,56%,6.67,20%,,0.00,0%,26%,0.0
7843,Roger Zapata,4,,-0,"5' 11""",170 lbs.,--,Southpaw,"May 09, 1986",2.60,51%,2.13,36%,,0.00,0%,81%,1.4


### -[done] saving file

In [15]:
df.to_csv('../data/ufc_fighters_stats.csv')

## >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>